![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_XLM_RoBERTa.ipynb)

# Import ONNX XLM-RoBERTa models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XLM-RoBERTa from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use XLM-RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- We install the `transformers` package (version ≥ 4.36.0) along with `optimum[onnxruntime]` (version ≥ 1.12.0), `torch`, `torchvision`, and `onnxruntime`.
- These are required for exporting and running Hugging Face models in ONNX format. While Spark NLP doesn't require ONNX, these tools are essential for preparing models before integration.
- The specified versions have been tested, but newer versions may also work.


In [ ]:
!pip install --quiet --upgrade \
    torch torchvision \
    "transformers>=4.36.0" \
    "optimum[onnxruntime]>=1.12.0" \
    onnxruntime

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the XLM-RoBERTa model, we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import XLMRobertaTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "xlm-roberta-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The model xlm-roberta-base was already converted to ONNX but got `export=True`, the model will be converted to ONNX once again. Don't forget to save the resulting model with `.save_pretrained()`


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

('onnx_models/xlm-roberta-base/tokenizer_config.json',
 'onnx_models/xlm-roberta-base/special_tokens_map.json',
 'onnx_models/xlm-roberta-base/sentencepiece.bpe.model',
 'onnx_models/xlm-roberta-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 1089044
-rw-r--r-- 1 root root        640 Jun 17 01:42 config.json
-rw-r--r-- 1 root root 1110092472 Jun 17 01:43 model.onnx
-rw-r--r-- 1 root root    5069051 Jun 17 01:43 sentencepiece.bpe.model
-rw-r--r-- 1 root root        280 Jun 17 01:43 special_tokens_map.json
-rw-r--r-- 1 root root       1203 Jun 17 01:43 tokenizer_config.json


- We need to move the `sentencepiece.bpe.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models with SentencePiece-based tokenizers.

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 4952
-rw-r--r-- 1 root root 5069051 Jun 17 01:43 sentencepiece.bpe.model


Voila! We have our `spiece.model` inside assets directory

## Import and Save XLM-RoBERTa in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 747.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 27.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `XlmRoBertaEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `XlmRoBertaEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import XlmRoBertaEmbeddings

xlm_roberta = XlmRoBertaEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("xlm_roberta")\
    .setCaseSensitive(True)\
    .setDimension(768)\
    .setStorageRef('xlm_roberta_base')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
xlm_roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX XLM-RoBERTa model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 1089204
drwxr-xr-x 2 root root       4096 Jun 17 01:59 metadata
-rw-r--r-- 1 root root 1110261996 Jun 17 02:00 xlmroberta_onnx
-rw-r--r-- 1 root root    5069051 Jun 17 02:00 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XLM-RoBERTa model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = XlmRoBertaEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("xlm_roberta")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    embeddings
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(xlm_roberta.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[0.09555595, 0.21...|
|[0.08841043, 0.18...|
|[0.0464, 0.122910...|
|[0.034001112, 0.1...|
|[-0.03272612, 0.0...|
|[-0.024419192, 0....|
|[0.01732055, 0.11...|
|[-0.08084225, 0.0...|
|[0.0010092389, 0....|
|[-0.00622857, 0.1...|
|[-0.051159456, 0....|
|[-0.073909596, 0....|
|[-0.040206, 0.061...|
|[0.017141085, 0.2...|
|[0.102552906, 0.2...|
|[0.08336049, 0.53...|
|[0.087454535, 0.2...|
|[0.037222322, 0.2...|
|[-0.028664015, 0....|
|[0.03084144, 0.20...|
+--------------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of XLM-RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀
